In [ ]:
# cell for importing packages for the first time
# import Pkg; Pkg.add("StaticArrays")
# import Pkg; Pkg.add("Revise")




In [1]:
using Plots

using StaticArrays


push!(LOAD_PATH, pwd())

module PhysConst
    export CL,Gr,QE,MSOL,MSUN,ARAD,SGMB,RGAS,
            PC,RSUN,RSUN,YR,MSOLYR,GAM53,KPE,PARSEC,KPE,KPD,M_MW
    CL = 2.997925E10;
    Gr = 6.67384E-8; 
    QE=4.80325E-10;
    MSOL = 1.989E33;
    MSUN = 1.989E33;

    ARAD = 7.56464E-15; 
    SGMB = ARAD*CL/4

    RGAS = 8.31E7; 
    PC = 3.085678E18;
    MP = 1.672661E-24; 
    M_U = 1.660531E-24
    RSOL = 6.95E10
    RSUN = 6.95E10
    YR = 365*24*3600.
    MSOLYR = 1.989e33/YR

    GAM = 5/3.

    KPE = 0.4
    PARSEC=3.0E18
    KPD = 10.

    M_MW=1.

    tst=1

end



Main.PhysConst

In [2]:

#-------------------------------------- STEP 1 -------------------------------------- #
# define necessary data structures

# module SpaceGrid


using StaticArrays
import Base.@kwdef

    @kwdef struct SGridPar
        
        Nfreq = 1 ; # ν - N bins 
        Nang = 10
    
        N_fre_ang = Nfreq*Nang

        Nactive = 10;
    
        nghost = 1 #N of ghost cells at every boundary.
            
        Nx = Nactive;   # N of active cells
        Ny = Nactive;
        Nz = Nactive;
        
        Nxtot = Nx +  2*nghost;
        Nytot = Ny +  2*nghost;
        Nztot = Nz +  2*nghost;
    
        is = 1 + nghost; 
        ie = Nxtot -nghost;        

        js = 1 + nghost;  
        je = Nytot - nghost;
        
        ks = 1 + nghost;
        ke = Nztot - nghost;
            
        x1s = -1.
        x1e = 1.
    
        x2s = -1.
        x2e = 1.

        x3s = -1.
        x3e = 1.
        
    end

    @kwdef struct RGridPar
        Nang = 2
    
        Nfreq = 1   
    
    end


# ------ define instances ----------

sg= SGridPar()

@show rad = RGridPar()

# ----------------------------------

x1 = Array{Float64}(undef, sg.Nxtot)
x2 = Array{Float64}(undef, sg.Nytot)
x3 = Array{Float64}(undef, sg.Nztot)

x1b = Array{Float64}(undef, sg.Nxtot)
x2b = Array{Float64}(undef, sg.Nytot)
x3b = Array{Float64}(undef, sg.Nztot)


@time ir = zeros(Float64, sg.N_fre_ang, sg.Nx, sg.Ny, sg.Nz); # radiation intensity



function MakeGrid(x1, x2, x3, x1b, x2b, x3b, sg)  #staggered grid

    # add check sizes
    
    xtmp = range(sg.x1s, sg.x1e; length = sg.Nxtot - 2sg.nghost)
    
#     @show sg.is, sg.ie sg.Nx sg.nghost size(xtmp) size(x1) size(sg.is:sg.ie) size(2:2)
    
    x1[sg.is:sg.ie] .= xtmp[:] 
    x1[sg.is-1] = 2*x1[sg.is]- x1[sg.is+1]
    x1[sg.ie+1] = 2*x1[sg.ie] -x1[sg.ie-1]

    xtmp = range(sg.x2s, sg.x2e; length = sg.Nytot - 2sg.nghost)    

    x2[sg.js:sg.je] .= xtmp[:]     
    x2[sg.js-1] = 2*x2[sg.js]- x2[sg.js+1]
    x2[sg.je+1] = 2*x2[sg.je] -x2[sg.je-1]

    xtmp = range(sg.x3s, sg.x3e; length = sg.Nztot - 2sg.nghost)    

    x3[sg.ks:sg.ke] .= xtmp[:]     
    x3[sg.ks-1] = 2*x3[sg.ks]- x3[sg.ks+1]
    x3[sg.ke+1] = 2*x3[sg.ke] -x3[sg.ke-1]
    
    
    x1b[1] = x1[1] - 0.5(x1[sg.is]-x1[1])    
    for i in sg.is:(sg.ie+1)
        x1b[i] = x1[i-1] + 0.5(x1[i]-x1[i-1])
    end

    x2b[1] = x2[1] - 0.5(x2[sg.js]-x2[1])    
    for j in sg.js:(sg.je+1)
        x2b[j] = x2[j-1] + 0.5(x2[j]-x2[j-1])
    end

    x3b[1] = x3[1] - 0.5(x3[sg.ks]-x3[1])    
    for k in sg.ks:(sg.ke+1)
        x3b[k] = x3[k-1] + 0.5(x3[k]-x3[k-1])
    end
    
    
    
end


MakeGrid(x1,x2,x3,x1b,x2b,x3b,sg)
@show x3
@show x3b

@kwdef mutable struct LongRay1
#     first time calculation of the long ray
    
    numOfElm::Int32
    
    norm = Vector{Float64}  #direction
    
    ijkOfCellCrossed:: Matrix{Int32}
    
#     dl :: Vector{Float64}
    
    xyzPos ::Matrix{Float64}

end


#  allocate one big array
@time begin
    
    NInRay = max(sg.Nxtot,sg.Nytot,sg.Nztot)

#     rayOne = LongRay1(10, zeros(Float64,3), Array{Int32}(undef,3,NInRay), zeros(Float64,NInRay), Array{Float64}(undef,3,NInRay) )

    rayOne = LongRay1(10, zeros(Float64,3), Array{Int32}(undef,3,NInRay),  Array{Float64}(undef,3,NInRay) )

    NraysPerFourPi = 3

    Ntot3 = sg.Nxtot*sg.Nytot*sg.Nztot




    arrayOfRays = Array{LongRay1, 4}(undef, (sg.N_fre_ang, sg.Nxtot, sg.Nytot, sg.Nztot));


    for m = 1:sg.N_fre_ang, i=1:sg.Nxtot, j=1:sg.Nytot, k=1:sg.Nztot
#   arrayOfRays[m,i,j,k] = LongRay1(10, zeros(Float64,3), Array{Int32}(undef,3,NInRay), zeros(Float64,NInRay), Array{Float64}(undef,3,NInRay) )
        
    arrayOfRays[m,i,j,k] = LongRay1(10, zeros(Float64,3), Array{Int32}(undef,3,NInRay), Array{Float64}(undef,3,NInRay) )


    end

    
end


# end



rad = RGridPar() = RGridPar(2, 1)
  0.013417 seconds (28.44 k allocations: 1.580 MiB)
x3 = [-1.2222222222222223, -1.0, -0.7777777777777778, -0.5555555555555556, -0.3333333333333333, -0.1111111111111111, 0.1111111111111111, 0.3333333333333333, 0.5555555555555556, 0.7777777777777778, 1.0, 1.2222222222222223]
x3b = [-1.3333333333333335, -1.1111111111111112, -0.8888888888888888, -0.6666666666666667, -0.4444444444444444, -0.2222222222222222, 0.0, 0.2222222222222222, 0.4444444444444444, 0.6666666666666667, 0.8888888888888888, 1.1111111111111112]
  0.027969 seconds (129.42 k allocations: 14.758 MiB, 29.21% gc time)


In [ ]:


# mutable struct SegmentData{
#  /*! cell data along the trajectory */
#   /*!  current index of cell along the ray, phi index is the same */
#   /*! if the source is symmetrical */
#   int i,k;    
#   float dl; //length element    
# }SegmentData;

# RayCellInd1 = Array{Int32}(undef,1)
# RayCellInd2 = Array{Int32}(undef,1)
# RayCellInd3 = Array{Int32}(undef,1)
# push!(RayCellInd1, 1)
# RayCellInd1 .=0
# RayCellInd2 .=0
# RayCellInd3 .=0


# create array of ray directions, mu, for every ijk cell (nx,ny,nz) components
# for every ray; 
# nang - number of rays per cell 



# RayData = Array of ( (iCross, jCross, kCross), NcrossCells,(istart, jstart, kstart)=3)
# NcrossCells=3
# @time RayData = zeros(3,  zeros(3),  3)





In [4]:

rad




RGridPar(2, 1)

In [3]:

module RadTransfer

    import LinearAlgebra
    const IX = 1
    const IY = 2
    const IZ = 3

    const BIGNUMBER = 10.0^50


    @time norm = zeros(Float64, rad.Nang, 3, sg.Nx, sg.Ny, sg.Nz);

    ijk_pos = Array{Int64}(undef,3)

    ijk_max = [ sg.ie+1, sg.je+1, sg.ke+1 ] #max ijk index 
    ijk_min = [ sg.is, sg.js, sg.ks ] #min index

    distToNextCell = fill(BIGNUMBER,3) #used in FirstTimeTraceGridOverOneLongRay()


function FirstTimeTraceGridOverOneLongRay(rayOne, sg)

    iter=1
    dist = 0.0

    # xc = 0

    xn = Array{Float64}(undef, 3) .= x3c

    itermax = 2sg.Nx

    for iter in 1:itermax #max length, may be smaller than actual

    # @show iter

      distToNextCell .= BIGNUMBER

        for (n_it, norm_i ) in enumerate(norm3) #QUESTION: maybe iter over pre-calclulated 1/norms


          xc = x3c[n_it] 

             if iter > 1 

                itmp = ijk_tmp[n_it] = ijk_pos[n_it] + copysign(1, norm3[n_it])

                @debug  " debug: (itmp, iter)= " , itmp, iter

                xn[n_it] = xbi_zip[n_it][itmp]                                            


             elseif iter==1 #first step from the middle of the cell

                if norm_i>0                
                   itmp = ijk_tmp[n_it] = ijk_pos[n_it] + copysign(1, norm3[n_it])                     
                elseif norm_i <0               
                    itmp = ijk_tmp[n_it] = ijk_pos[n_it]  #belongs to the same index                                
                else                
                    itmp = ijk_pos[n_it]            
                end

                xn[n_it] = xbi_zip[n_it][itmp]

             end


             if norm_i != 0                      
                distToNextCell[n_it] = abs((xn[n_it] - xc)/norm3[n_it])
             else
                distToNextCell[n_it]=BIGNUMBER
             end


        #println("iter= ",iter," xn=", xn, " ===> ", "distToNextCell[",n_it,"]=",distToNextCell[n_it])



        end #for loop over norm_i     

            posIndxToUpdate = argmin(distToNextCell)
            dist = distToNextCell[posIndxToUpdate]


        ijk_pos[posIndxToUpdate] = ijk_tmp[posIndxToUpdate] #update only relevant one
    

        #println("ijk_pos[posIndxToUpdate = ", posIndxToUpdate, "]",  "  =  ",   ijk_pos  )


        xn .= x3c + norm3*dist
        x3c .= xn

        #update rayOne
        rayOne.numOfElm = iter
        rayOne.ijkOfCellCrossed[:,iter] .= ijk_pos
        rayOne.xyzPos[:,iter] .= xn

        println("xn=", xn)

        if ijk_pos[posIndxToUpdate] == ijk_max[posIndxToUpdate] || ijk_pos[posIndxToUpdate] == ijk_min[posIndxToUpdate] || ijk_pos[posIndxToUpdate] == 1 || iter > 100 

            break; 
        end      


    end #over ray

end # FirstTimeTraceGridCalcOneLongRay



norm[1, IX, :, :, :] .=  1
norm[1, IY, :, :, :] .= -1
norm[1, IZ, :, :, :] .= -1


iOf_0 = argmin(abs.(x1)); jOf_0 = argmin(abs.(x2)); kOf_0 = argmin(abs.(x3))
# ijk_pos .= [sg.is, sg.js, sg.ks]
# ijk_pos .= [iOf_0, div(sg.je,2), div(sg.ke,2)]
# ijk_pos .= [sg.ie, sg.je, sg.ke]
ijk_pos .= [iOf_0,  jOf_0,  kOf_0]

ijk_tmp = [0, 0, 0]


norm3 = [norm[1, IX, 1, 1, 1], norm[1, IY, 1, 1, 1] , norm[1, IZ, 1, 1, 1]]


norm3=LinearAlgebra.normalize(norm3)
@show norm3

rayOne.norm = norm3 #update ray direction


# should replace 1/norm just in case when Float norm==0.
#          one2norm_i = max(abs())

xbi_zip = [x1b, x2b, x3b]



@show ijk_pos
x3c = [ x1[ijk_pos[IX]], x2[ijk_pos[IY]], x3[ijk_pos[IZ]] ]
@show x3c

@show x1
@show x1b

@show x2
@show x2b

@show x3
@show x3b
# @show xbi_zip[2]


RadTransfer.FirstTimeTraceGridOverOneLongRay(rayOne, sg)



# PlotRay(rayOne, sg)


# @show ijk_pos size(rayOne.ijkOfCellCrossed[:][3])
# @show rayOne.numOfElm
# @show rayOne.ijkOfCellCrossed
# @show rayOne.xyzPos
# inline()



# display(plt)

# function PlotRay(ray, sg)
#     println( ray.numOfElm)    
#     x = ray.xyzPos[1,1:ray.numOfElm]
#     y = ray.xyzPos[2,1:ray.numOfElm]   
#     z = ray.xyzPos[3,1:ray.numOfElm]                 
#     plot3d(x,y,z,
#         xlabel = "x", ylabel = "y", zlabel = "z", 
#         xlim= (sg.x1s,sg.x1e),ylim= (sg.x2s,sg.x2e),zlim= (sg.x3s,sg.x3e))    
# end


println("done")

end #module RadTransfer

UndefVarError: UndefVarError: rad not defined

In [5]:
#-------------------------------------- STEP 2 -------------------------------------- #



# @show size(norm);



  0.000016 seconds (4 allocations: 47.047 KiB)
norm3 = [0.5773502691896258, -0.5773502691896258, -0.5773502691896258]
ijk_pos = [6, 6, 6]
x3c = [-0.1111111111111111, -0.1111111111111111, -0.1111111111111111]
x1 = [-1.2222222222222223, -1.0, -0.7777777777777778, -0.5555555555555556, -0.3333333333333333, -0.1111111111111111, 0.1111111111111111, 0.3333333333333333, 0.5555555555555556, 0.7777777777777778, 1.0, 1.2222222222222223]
x1b = [-1.3333333333333335, -1.1111111111111112, -0.8888888888888888, -0.6666666666666667, -0.4444444444444444, -0.2222222222222222, 0.0, 0.2222222222222222, 0.4444444444444444, 0.6666666666666667, 0.8888888888888888, 1.1111111111111112]
x2 = [-1.2222222222222223, -1.0, -0.7777777777777778, -0.5555555555555556, -0.3333333333333333, -0.1111111111111111, 0.1111111111111111, 0.3333333333333333, 0.5555555555555556, 0.7777777777777778, 1.0, 1.2222222222222223]
x2b = [-1.3333333333333335, -1.1111111111111112, -0.8888888888888888, -0.6666666666666667, -0.4444444444444444

UndefVarError: UndefVarError: x3c not defined

In [ ]:
# logger=Logging.SimpleLogger(stderr,Logging.Debug)

function TestOneRay(ray, sg)
    xyz_s=Vector{Float64}(undef,3)    
    xyz_e=Vector{Float64}(undef,3)
    
    xyz_0e=Vector{Float64}(undef,3)
    
    xyz_s .= ray.xyzPos[:,1]
    
    xyz_e = ray.xyzPos[:,ray.numOfElm]

     dist =LinearAlgebra.norm( xyz_e[:].-xyz_s[:]) 
    
    xyz_0e = xyz_s + ray.norm*dist
    
    
    @show xyz_e, xyz_0e, dist
    
end


TestOneRay(rayOne, sg)



In [ ]:
module debugByPlot

using Plots
# x= LinRange(0,30,10)

# import 

x = [0.5, 1.]
y = [0., 1.]
z = [0., 0.1];


plt=plot3d(x,y,z)



end


In [ ]:
# function mtest()
# L = M = N = Int(5e2);

# vec_vec_vec = Array{Array{Vector}}(L);

# for i = 1:L vec_vec_vec[i] = [zeros(N) for j = 1:M] end;

# vec_mat = Array{Matrix}(L);

# fill!(vec_mat, rand(M, N));

# arr = rand(L, M, N);

# @time  for i = 1:L for j = 1:M for k = 1:N vec_vec_vec[i][j][k] += 1; end; end; end; 

# @time  for i = 1:L for j = 1:M for k = 1:N vec_mat[i][j,k] += 1; end; end; end; 

# @time  for k = 1:N for j = 1:M for i = 1:L arr[i,j,k] += 1; end; end; end; 

# end
# mtest()

In [ ]:
copysign(1, 0)

In [ ]:
import Pkg; Pkg.add("Plotly")

In [ ]:
# using Plotly
# default(show = true)
# plotly()
